Особая тонкость: последний случай нельзя проверить регрессией из sklearn, потому что мы минимизируем норму **всех** весов в том числе того, что без признака, а встроенная регерессия пользуется регуляризацией, где не учитывается вес без признака. Для проверки работы можете использовать `LinearRegression(fit_intercept=False)` и сравнить ее со своей, предварительно убрав из нее вес без признака.

# SLIDE (1) Свертка тензора

Вам на вход подается целочисленный тензор 3-го измерения. Сначала возьмите перемножение из всех чисел по $0$ и $2$ координатам. А потом среди оставшихся чисел возьмите минимум.
### Sample
#### Input:
```python
X = np.array([[[ 1, 2, 3],
               [ 4, 5, 6],
               [ 7, 8, 9]],
              
              [[ 1, 2, 3],
               [ 4, 5, 6],
               [ 7, 8, 9]]])
```
#### Output:
```python
36
```

# TASK

In [ ]:
#precondition
assert_no_tokens(['print', 'while', 'map', 'for', 'open'])

In [1]:
import numpy as np

def max_result(X: np.ndarray) -> np.float:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [2]:
def max_result(X: np.ndarray) -> float:
    return X.prod(axis=(0,2)).min()

In [5]:
######################################################
assert_equal(
    max_result(np.array([[[ 1, 2, 3],
                          [ 4, 5, 6],
                          [ 7, 8, 9]],
                         [[ 1, 2, 3],
                          [ 4, 5, 6],
                          [ 7, 8, 9]]])),
    36
)
######################################################
assert_equal(
    max_result(np.array([[[ 0, 0],
                          [ 0, 0],
                          [ 0, 0]],
                         [[ 0, 0],
                          [ 0, 0],
                          [ 0, 0]]])),
    0
)
######################################################
assert_equal(
    max_result(np.array([[[ 1,  1],
                          [ 1,  1]],
                         [[ 1,  1],
                          [ 1,  1]]])),
    1
)
######################################################
assert_almost_equal(
    max_result(np.array([[[  5.0,  -3.2],
                          [42.27, -27.4]],
                         [[  4.2,   5.2],
                          [  -42,  2.42]]])),
    -349.440000,
    decimal=6
)
######################################################
assert_equal(
    max_result(np.array(
     [[[ 3,  3,  0,  2,  6],
       [24, 13,  0, 13,  1],
       [19,  2,  0, 18,  1],
       [18,  8,  0, 22, 12],
       [25, 25,  0, 25, 19]],
      
      [[ 3,  3,  0,  2,  6],
       [24, 13,  0, 13,  1],
       [19,  2,  0, 18,  1],
       [18,  8,  0, 22, 12],
       [25, 25,  0, 25, 19]]
     ])),
    0
)
######################################################
assert_equal(
    max_result(np.array(
     [[[ 3,  3,  5,  2,  6],
       [24, 13,  7, 13,  1],
       [19,  2,  4, 18,  1],
       [18,  8,  6, 22, 12],
       [25, 25,  7, 25, 19]],
      
      [[ 3,  3,  12,  2,  6],
       [24, 13,  12, 13,  1],
       [19,  2,  54, 18,  1],
       [18,  8,  78, 22, 12],
       [25, 25,  42, 25, 19]]
     ])),
    699840
)

# SLIDE (2) Мой GridSearch

Вам необходимо реализовать собственный простой гридсерч - перебор параметров с поиском наилучшего для данного эстиматора.

Она принимает на вход: 
* `estimator`, для которого ищем наилучшие параметры
* `param_grid` - словарь параметров и их возможных значений, по которым будем перебирать модели
* `cv` ($k$) - объект разбиения `KFold` или количество фолдов, на которые мы бьем данные (по умолчанию 3)
* `scoring` - функция метрики, по которой оцениваем полученные результаты (замечание: в реальной `cross_val_score` здесь будет стоять объект `scorer`, у нас же будет стоять функция, например `metrics.accuracy_score`)

По сути, вам достаточно реализовать только функцию `fit`, но никто не запрещает добавлять доп. функции и менять что-то в `init` и `predict`

При вызове `fit` вам нужно найти 2 атрибута:
* `cv_results_` - словарь параметров
* `best_params` - словарь наилучших параметров (пример `{max_depth: 4, min_samples_leaf: 5}`)

В нашем `cv_results_` будет только 2 колонки:
* `params` - словарь с параметрами вида: `{max_depth: 4, min_samples_leaf: 5}`
* `mean_test_score` - среднее значение кроссвалидации


### Sample
#### Input:
```python
estimator = LinearRegression()
cv = 2
scoring = sklearn.metrics.r2_score


```
#### Output:
```python
array([0.75, 0.83673469])
```

In [44]:
class MYGridSearch:
    def __init__(self, estimator, param_grid, cv, scoring):
        self.estimator = estimator
        self.grid_params = param_grid
        self.scoring = scoring
        self.cv = cv
        
        self.cv_results = {'params': []
                           'mean_rest_score': []} # [{'params': _, 'mean_test_score': _}]
        self.best_params = {}
        
    def fit(self, X, y):
        """
        Найдите лучшие значения и обучитесь на них
        """
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        self.estimator.set_params(self.best_params)
        self.estimator.fit(X, y)
        return self
    
    def predict(self, X, y=None):
        return self.estimator.predict(X, y)

In [40]:
#Авторское решение
class GridSearchCV:
    def __init__(self, estimator, grid_params, cv=3, scoring=accuracy_score):
        self.estimator = estimator
        self.grid_params = grid_params
        self.scoring = scoring
        self.cv = cv
        self.cv_results = [] # [{'params': _, 'mean_test_score': _}]
        self.best_params = {}
        
    def get_params_product_list(self):
        items = sorted(self.grid_params.items())
        keys, values = zip(*items)
        for v in product(*values):
            params = dict(zip(keys, v))
            yield params
        
    def fit(self, X, y):
        best_score = 0.0
        for params in get_params_product_list():
            self.estimator.set_params(params)
            cv_score = cross_val_score(self.estimator, X, y, n_splits=self.cv, scoring=self.scoring)
            mean_score = np.mean(cv_score)
            if mean_score > best_score:
                best_score = mean_score
                self.best_params = params

            self.cv_results['params'].append(params)
            self.cv_results['mean_test_score'].append(mean_score)
            
        #refit
        self.estimator.set_params(self.best_params)
        self.estimator.fit(X, y)
        return self
    
    def predict(self, X, y=None):
        return self.estimator.predict(X, y)

# SLIDE (2) Первая классификация

Мы взяли [известные данные](https://www.kaggle.com/uciml/pima-indians-diabetes-database) для задачи бинарной классификации и разбили их произвольным образом на тренировачную и тестовую выборки в отношении 4:1. 

$Y$ в этих данных выступает столбик `Outcome`, в качаестве $X$ - все остальное. 

Вам на вход подается **тренировачная** выборка. Вы можете используя любой классификатор, как угодно изменяя гиперпараметры, попытаться получить на тестовых данных на сервере долю правильных ответов больше $0.7$. В задаче также необходимо вернуть подходящую обученную модель. Доля правильных ответов, это количество совпаших ответов, к количеству несовпавших ответов. Подробнее: [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html).

Алгоритм решения:
* Скачиваете данные по ссылке
* Изучаете данные, может как-то изменяете их
* Сами разделяете выборку на тренировачную и тестовую с помощью `train_test_split`
* Подбираете алгоритм, чтобы он давал необходимый (и даже с запасом) результат при различных разбиениях: изменяйте параметр `random_state` в `train_test_split`
* Для нахождения доли правильных ответов воспользуйтесь `sklearn.metrics.accuracy_score`
* В самой функции можете как-то изменить данные
* Вызываете необходимый алгоритм и обучаете его на входных данных внутри функции
* Верните обученную модель в функции

In [2]:
import numpy as np
import pandas as pd

def classification(X_train: np.ndarray, y_train: np.ndarray):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [3]:
def classification(X_train: np.ndarray, y_train: np.ndarray):
    return LogisticRegression().fit(X_train, y_train)

In [4]:
from sklearn.metrics import accuracy_score

data = pd.read_csv('diabetes.csv')
y = data['Outcome']
X = data.drop(columns=['Outcome']).values
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
y_pred = classification(X_train, y_train).predict(X_test)
assert accuracy_score(y_pred, y_test) > 0.7
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
y_pred = classification(X_train, y_train).predict(X_test)
assert accuracy_score(y_pred, y_test) > 0.7
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
y_pred = classification(X_train, y_train).predict(X_test)
assert accuracy_score(y_pred, y_test) > 0.7
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91)
y_pred = classification(X_train, y_train).predict(X_test)
assert accuracy_score(y_pred, y_test) > 0.7
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)
y_pred = classification(X_train, y_train).predict(X_test)
assert accuracy_score(y_pred, y_test) > 0.7
######################################################

FileNotFoundError: [Errno 2] File b'diabetes.csv' does not exist: b'diabetes.csv'

# SLIDE (1) Регрессия и kaggle

А вот мы добрались и до [соревнования](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)! Вам нужно решить задачу регрессии и предсказать цену на покупку дома. 

Пока мы ограничены в возможностях вам дается урезанная задача: мы оставили только численные признаки `df._get_numeric_data().fillna(0)` и разбили данные произвольным образом на тренировачную и тестовую выборки в отношении 4:1. $y$ в этих данных выступает столбик `SalePrice`, в качаестве $X$ - все остальное. 

Вам на вход подается только **тренировачная** выборка. Вы можете используя любой регрессор, как угодно изменяя гиперпараметры, попытаться получить на тестовых данных на сервере результат `RMSLE` между вашим ответом и правильным **меньше** $0.3$. В задаче также необходимо вернуть подходящую обученную модель. 

`RMSLE` - root mean squared log error или 
$$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(y^i_{pred}) - log(y^i_{real}))^2}$$

Подробнее: [MSLE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_log_error.html).

Алгоритм решения:
* Скачиваете данные по ссылке и загружаете в `DataFrame`
* Вызываете команду `df._get_numeric_data().fillna(0)` чтобы оставить только численные данные и заполнить Nan-ы 0-ми.
* Изучаете данные, может как-то изменяете их
* Сами разделяете выборку на тренировачную и тестовую с помощью `train_test_split`
* Подбираете алгоритм, чтобы он давал необходимый (и даже с запасом) результат при различных разбиениях: изменяйте параметр `random_state` в `train_test_split`
* Для нахождения `RMSLE` воспользуйтесь `np.sqrt(sklearn.metrics.mean_squared_log_error(.,.))`
* В самой функции как-то можете изменить данные
* Вызываете необходимый алгоритм и обучаете его на входных данных внутри функции
* Верните обученную модель в функции

In [15]:
import numpy as np
import pandas as pd

def regression(X: np.ndarray, y:np.ndarray):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [ ]:
def regression(X: np.ndarray, y:np.ndarray):
    return RandomForestRegressor(max_depth= 8, n_estimators = 150).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_log_error

data = pd.read_csv('houseprice/train.csv')
y = data['SalePrice']
X = data._get_numeric_data().fillna(0).drop(columns=['SalePrice']).values
######################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
y_pred = regression(X_train, y_train).predict(X_test)
assert np.sqrt(mean_squared_log_error(y_pred, y_test)) < 0.3
######################################################

In [491]:
df = pd.read_csv('houseprice/train.csv')

In [460]:
df2 = pd.read_csv('houseprice/test.csv')._get_numeric_data().fillna(0)

In [461]:
data = df2._get_numeric_data().fillna().values

In [498]:
data = df._get_numeric_data().dropna(axis=0)

In [499]:
y = data['SalePrice']
X = data.drop(columns=['SalePrice']).values

In [500]:
X.shape, y.shape

((1121, 37), (1121,))

In [410]:
from sklearn.linear_model import LinearRegression

In [414]:
model = LinearRegression().fit(X,y)

In [416]:
model.predict(X)

array([227286.82182012, 196557.96053216, 222772.94061721, ...,
       223960.77437461, 131513.95653512, 152162.43258722])

In [419]:
from sklearn import metrics

In [501]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [502]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

lm = RandomForestRegressor(max_depth= 10, n_estimators = 150)

lm.fit(X_train, y_train)

# model evaluation
lm_yhat = lm.predict(X_test)

print("rmsle: ")
print(np.sqrt(metrics.mean_squared_error(np.log(y_test), np.log(lm_yhat))))

rmsle: 
0.16816613729199362


In [482]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

lm = LinearRegression()

lm.fit(X_train, y_train)

lm_yhat = lm.predict(X_test)
lm_yhat[lm_yhat < 0] = 0

print("rmsle: ")
print(np.sqrt(metrics.mean_squared_error(y_test, lm_yhat)))

rmsle: 
36547.443679754775


In [441]:
lm.predict(X_real_test._get_numeric_data().fillna(0))

array([-18722.45784953])

In [417]:
cross_val_score(X, y, LinearRegression(), cv=5)

TypeError: estimator should be an estimator implementing 'fit' method, array([[1.000e+00, 6.000e+01, 6.500e+01, ..., 0.000e+00, 2.000e+00,
        2.008e+03],
       [2.000e+00, 2.000e+01, 8.000e+01, ..., 0.000e+00, 5.000e+00,
        2.007e+03],
       [3.000e+00, 6.000e+01, 6.800e+01, ..., 0.000e+00, 9.000e+00,
        2.008e+03],
       ...,
       [1.458e+03, 7.000e+01, 6.600e+01, ..., 2.500e+03, 5.000e+00,
        2.010e+03],
       [1.459e+03, 2.000e+01, 6.800e+01, ..., 0.000e+00, 4.000e+00,
        2.010e+03],
       [1.460e+03, 2.000e+01, 7.500e+01, ..., 0.000e+00, 6.000e+00,
        2.008e+03]]) was passed

In [50]:
import numpy as np
import pandas as pd

def regression(X: np.ndarray, y:np.ndarray):
    pass

In [35]:
import inspect
lines = inspect.getsource(regression)
assert ' print(' not in lines
assert ' open(' not in lines
######################################################
np.random.seed(228)
n = 200
a = np.random.normal(loc=0, scale=1, size=(n, 2)) #первый класс
b = np.random.normal(loc=3, scale=2, size=(n, 2)) #второй класс
X_clf = np.vstack([a, b]) #двумерный количественный признак
y_clf = np.hstack([np.zeros(n), np.ones(n)]) #бинарный признак

model = fit_gs(X_clf, y_clf)

df = pd.DataFrame(model.cv_results_)
params = df[df['rank_test_score'] == 1].reset_index(drop=True).loc[0]['params']
assert params['max_depth'] == 3
assert params['n_estimators'] == 11


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 3, 'n_estimators': 11}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [49]:
def find_rule(x_train, y_train):
    def entropy(y):
        n = len(y)
        p0 = len(y[y == 0]) / n
        p1 = len(y[y == 1]) / n
        if p0 == 0 or p1 == 0:  # when there is only one class in the group, entropy is 0
            return 0
        return -p0 * np.log2(p0) - p1 * np.log2(p1)

    def ig(x_train, y_train, threshold):
        group0 = y_train[x_train <= threshold]
        group1 = y_train[x_train > threshold]
        n = len(y_train)
        n0 = len(group0)
        n1 = len(group1)
        return entropy(y_train) - (n0 / n) * entropy(group0) - (n1 / n) * entropy(group1)
    
    best_t = x_train[-1]
    best_score = 0
    for t in np.unique(x_train)[:-1]:
        cur_score = ig(x_train, y_train, t)
        if cur_score > best_score:
            best_t = t
            best_score = cur_score
    return best_t, best_score

# SLIDE (1) ID3 Decision Tree

ID3(Таблица примеров, Целевой признак, Признаки)

    Если все примеры положительны, то возвратить узел с меткой «+».
    Если все примеры отрицательны, то возвратить узел с меткой «-».
    Если множество признаков пустое, то возвратить узел с меткой, которая больше других встречается в значениях целевого признака в примерах.
    Иначе:
        A — признак, который лучше всего классифицирует примеры (с максимальной информационной выгодой).
        Создать корень дерева решения; признаком в корне будет являться A {\displaystyle A} A.
        Для каждого возможного значения A {\displaystyle A} A ( v i {\displaystyle v_{i}} v_{i}):
            Добавить новую ветвь дерева ниже корня с узлом со значением A = v i {\displaystyle A=v_{i}} A=v_{i}
            Выделить подмножество E x a m p l e s ( v i ) {\displaystyle Examples(v_{i})} Examples(v_{i}) примеров, у которых A = v i {\displaystyle A=v_{i}} A=v_{i}.
            Если подмножество примеров пусто, то ниже этой новой ветви добавить узел с меткой, которая больше других встречается в значениях целевого признака в примерах.
            Иначе, ниже этой новой ветви добавить поддерево, вызывая рекурсивно ID3( E x a m p l e s ( v i ) {\displaystyle Examples(v_{i})} Examples(v_{i}), Целевой признак, Признаки)
    Возвратить корень.

In [ ]:
from sklearn.base import BaseEstimator
class DecisionTreeClassifier(BaseEstimator):
    def __init__(self, max_depth):
        self.depth_ = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
    """
    x: Feature set
    y: target variable
    par_node: will be the tree generated for this x and y. 
    depth: the depth of the current layer
    """
    if par_node is None:   # base case 1: tree stops at previous level
        return None
    elif len(y) == 0:   # base case 2: no data in this group
        return None
    elif self.all_same(y):   # base case 3: all y is the same in this group
        return {'val':y[0]}
    elif depth >= self.max_depth:   # base case 4: max depth reached 
        return None
    # Recursively generate trees! 
        # find one split given an information gain 
    col, cutoff, entropy = self.find_best_split_of_all(x, y)   
    y_left = y[x[:, col] < cutoff]  # left hand side data
    y_right = y[x[:, col] >= cutoff]  # right hand side data
    par_node = {'col': iris.feature_names[col], 'index_col':col,
                'cutoff':cutoff,
                'val': np.round(np.mean(y))}  # save the information 
    # generate tree for the left hand side data
    par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)   
    # right hand side trees
    par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)  
    self.depth += 1   # increase the depth since we call fit once
    self.trees = par_node  
    return par_node
    
def all_same(self, items):
    return all(x == items[0] for x in items)
    
    def predict(X_test):
        pass

In [4]:
x_train = np.arange(20)
y_train = np.array([0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1])

In [23]:
model.tree_.impurity

array([0.99277445, 0.9612366 , 0.59167278])

In [5]:
from sklearn.tree import DecisionTreeClassifier as DTC
model = DTC(criterion='entropy', max_depth=1).fit(x_train[:,np.newaxis], y_train)

In [57]:
model.tree_.threshold

array([12.5, -2. , -2. ])

In [65]:
def create_tree_image(clf):
    from sklearn.tree import export_graphviz
    export_graphviz(clf, out_file='tree.dot', feature_names = ['x'],
                    class_names = np.array(['0','1']),
                    rounded = True, proportion = False, precision = 2, filled = True)
    from subprocess import call
    call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

    # Display in python
    plt.figure(figsize = (9, 7))
    plt.imshow(plt.imread('tree.png'))
    plt.axis('off');
    plt.show()

In [67]:
import matplotlib.pyplot as plt
create_tree_image(model)

<Figure size 900x700 with 1 Axes>

In [5]:
p = 0
1e-15 * np.log(p + (1e-15))

-3.453877639491069e-14

# SLIDE (1) Momentum.

Один из недостатков sgd состоит в том, что он может не доходить до локального оптимального решения, а осциллировать в окрестности. 

![](http://sebastianruder.com/content/images/2015/12/without_momentum.gif)

Для решения этой проблемы существуют методы, позволяющие устранить этот недостаток, а также ускорить сходимость. Рассмотрим некоторые из них.

![](http://nghenglim.github.io/images/2015061300.png)

### Momentum
Этот метод позволяет направить sgd в нужной размерности и уменьшить осцилляцию. 

В общем случае он будет выглядеть следующим образом: 

$$ v_t = \gamma v_{t - 1} + \eta \nabla_{\theta}{J(\theta)}$$
$$ \theta = \theta - v_t$$

где

 - $\eta$ — learning rate
 - $\theta$ — вектор параметров (в нашем случае — $w$)
 - $J$ — оптимизируемый функционал
 - $\gamma$ — momentum term (обычно выбирается 0.9)


# TASK

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDMomentum(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000):
        self.gradient = gradient
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        v = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            # пересчитайте веса в стохаистическом градиентном спуске
            '''
            .∧＿∧ 
            ( ･ω･｡)つ━☆・*。 
            ⊂  ノ    ・゜+. 
            しーＪ   °。+ *´¨) 
                    .· ´¸.·*´¨) 
                    (¸.·´ (¸.·'* ☆  <YOUR CODE>
            '''
            self.w = 
        return self

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDMomentum(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000):
        self.gradient = gradient
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        v = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            cur_grad = self.gradient(self.w, X[index, :], np.array(y[index]), self.l)
            v = self.gamma * v + self.lr * cur_grad
            nw = self.w - v
            if np.linalg.norm(self.w - nw) < 1e-10:
                break
            self.w = nw
        return self

In [ ]:
X = np.array([[ 2.67973871e-01, -9.43407158e-01,  4.59566449e-01,
         1.63136986e-01, -5.44506820e-01]])
y = np.array([-1])

r0 = SGDMomentum(5, lambda a, b, c, d: a, max_iter=10, l=1, lr=1)
r0.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r0.fit(X, y)
r1 = SGDMomentum(5, lambda a, b, c, d: a, max_iter=10, l=1, lr=0.1)
r1.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r1.fit(X, y)
######################################################
assert np.allclose(r0.w.reshape(6), np.array([ 0.01753165,  0.0350633,   0.05259494,  0.0350633,   0.01753165, -0.01753165]))
assert np.allclose(r1.w.reshape(6), np.array([-0.05887894, -0.11775788, -0.17663682, -0.11775788, -0.05887894,  0.05887894]))
######################################################

# SLIDE (2) Adagrad.

Одной из сложностей является выбор размера шага (*learning rate*). Основное отличие данного метода от SGD состоит в том что размер шага определяется для каждого параметра индивидуально. Этот метод хорошо работает с разреженными данными большого объема. 

Обозначим градиент по параметру $\theta_i$ на итерации $t$ как $g_{t,i} = \nabla_{\theta}J(\theta_i)$. 

В случае sgd обновление параметра $\theta_i$ будет выглядеть следующим образом:

$$ \theta_{t+1, i} = \theta_{t, i} - \eta \cdot g_{t,i}$$

А в случае Adagrad общий шаг $\eta$ нормируется на посчитанные ранее градиенты для данного параметра:

$$ \theta_{t+1, i} = \theta_{t, i} - \dfrac{\eta}{\sqrt{G_{t,ii} + \varepsilon}} \cdot g_{t,i}$$

где $G_t$ — диагональная матрица, где каждый диагональный элемент $i,i$ — сумма квадратов градиентов для $\theta_{i}$ до $t$-ой итерации. $\varepsilon$ — гиперпараметр, позволяющий избежать деления на 0 (обычно выбирается около *1e-8*).

Так как матрица $G_t$ диагональна, в векторном виде это будет выглядеть следующим образом (здесь $\odot$ — матричное умножение):

$$ \theta_{t+1} = \theta_{t} - \dfrac{\eta}{\sqrt{G_t + \varepsilon}} \odot g_t $$

# TASK

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDAdagrad(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000, eps=1e-8):
        self.gradient = gradient
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.eps = eps # используйте его для пересчёта весов
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        v = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            # пересчитайте веса в стохаистическом градиентном спуске
            '''
            .∧＿∧ 
            ( ･ω･｡)つ━☆・*。 
            ⊂  ノ    ・゜+. 
            しーＪ   °。+ *´¨) 
                    .· ´¸.·*´¨) 
                    (¸.·´ (¸.·'* ☆  <YOUR CODE>
            '''
            self.w = 
        return self

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDAdagrad(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000, eps=1e-8):
        self.gradient = gradient
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.eps = eps # используйте его для пересчёта весов
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        G = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            cur_grad = self.gradient(self.w, X[index, :], np.array(y[index]), self.l)
            G += cur_grad ** 2
            nw = self.w - self.lr / np.sqrt(G + self.eps) *  cur_grad
            if np.linalg.norm(self.w - nw) < 1e-10:
                break
            self.w = nw
        return self

In [ ]:
X = np.array([[ 2.67973871e-01, -9.43407158e-01,  4.59566449e-01,
         1.63136986e-01, -5.44506820e-01]])
y = np.array([-1])

r0 = SGDAdagrad(5, lambda a, b, c, d: a * c, max_iter=5, l=1, lr=1)
r0.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r0.fit(X, y)
r1 = SGDAdagrad(5, lambda a, b, c, d: a * c, max_iter=5, l=1, lr=0.1)
r1.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r1.fit(X, y)
######################################################
assert np.allclose(r0.w.reshape(6), np.array([4.46637091, 4.53067376, 4.59204786, 4.53067376, 4.46637091, -4.46637091]))
assert np.allclose(r1.w.reshape(6), np.array([0.50417066, 0.58148037, 0.66822661, 0.58148037, 0.50417066, -0.50417066]))
######################################################

In [ ]:
# SLIDE (1) Momentum.